In [1]:
import pandas as pd

In [2]:
import os
os.getcwd()

'C:\\Users\\LENOVO\\DataAnalyticsProject\\data'

DATA PROFILLING = CUSTOMERS DATA STARTS

In [3]:
customers=pd.read_csv("customers.csv")

In [4]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Customer_ID     4 non-null      int64 
 1   Name            4 non-null      object
 2   Premium_Amount  4 non-null      int64 
 3   Claim_Amount    4 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 256.0+ bytes


In [5]:
customers.head()

,Customer_ID,Name,Premium_Amount,Claim_Amount
0,101,Alice,1000,200
1,102,Bob,1500,500
2,103,Charlie,1200,0
3,104,David,1300,300


In [6]:
customers.tail(5)

,Customer_ID,Name,Premium_Amount,Claim_Amount
0,101,Alice,1000,200
1,102,Bob,1500,500
2,103,Charlie,1200,0
3,104,David,1300,300


In [7]:
customers.sample(2)

,Customer_ID,Name,Premium_Amount,Claim_Amount
1,102,Bob,1500,500
3,104,David,1300,300


DATA CLEANING = CUSTOMERS DATA STARTS

In [8]:
customers.columns = customers.columns.str.strip().str.lower().str.replace(" ", "_")


In [9]:
for col in customers.select_dtypes(include="object").columns:
    customers[col]=customers[col].str.strip()

In [10]:
customers["name"]=customers["name"].fillna("UnKnown")

In [11]:
numeric_cols=["customer_id","premium_amount","claim_amount"]
for col in numeric_cols:
    customers[col]=pd.to_numeric(customers[col],errors="coerce").fillna(0)

In [12]:
customers=customers.drop_duplicates()

DATA MODELLING = CUSTOMERS DATA STARTS

In [13]:
#profit per customer
customers["profit"]= customers["premium_amount"]-customers["claim_amount"]

In [14]:
#Claim Ratio
customers["claim_ratio"]=customers["claim_amount"]/customers["premium_amount"]

In [15]:
#risk classify
def classify_risk(x):
    if x<=0.3:
        return "Low Risk"
    elif x<=0.8:
        return "Medium Risk"
    else:
        return "High Risk"
customers["risk_level"]=customers["claim_ratio"].apply(classify_risk)

In [16]:
#classify premium members
customers["premium_category"]=customers["premium_amount"].apply(
    lambda x:"High_Value" if x>=1500 else "Standard"
)

In [17]:
#finding negative values among the initial values
(customers[["premium_amount","claim_amount"]]<0).sum()

premium_amount    0
claim_amount      0
dtype: int64

In [18]:
#claims more than premium
(customers["claim_amount"]>customers["premium_amount"]).sum()

np.int64(0)

In [19]:
#Missing customerID
customers["customer_id"].isnull().sum()

np.int64(0)

In [20]:
customers.head()

,customer_id,name,premium_amount,claim_amount,profit,claim_ratio,risk_level,premium_category
0,101,Alice,1000,200,800,0.200000,Low Risk,Standard
1,102,Bob,1500,500,1000,0.333333,Medium Risk,High_Value
2,103,Charlie,1200,0,1200,0.000000,Low Risk,Standard
3,104,David,1300,300,1000,0.230769,Low Risk,Standard


CLEANED AND MODELLED CUSTOMERS DATA = EXPORT STARTS

In [21]:
customers.to_excel("outputs/DimCustomer.xlsx",index=False)
#SINCE THE CUSTOMERS DATA ANSWERS THE QUESTION WHO/WHAT TYPE OF QUESTIONS ALTHOUGH IT HAS NUMERICAL VALUES IT IS CONSIDERED AS DIMENSIONAL TABLE SO WE CAN SAVE IT AS DIMCUSTOMERS 

DATA PROFILLING = POLICIES DATA STARTS

In [22]:
policies= pd.read_excel("Policies.xlsx")

In [23]:
policies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Policy_ID    4 non-null      int64 
 1   Policy_Type  4 non-null      object
 2   Start_Date   4 non-null      object
 3   End_Date     4 non-null      object
dtypes: int64(1), object(3)
memory usage: 256.0+ bytes


In [24]:
policies.head()

,Policy_ID,Policy_Type,Start_Date,End_Date
0,201,Life,2023-01-01,2024-01-01
1,202,Health,2023-02-01,2024-02-01
2,203,Auto,2023-03-01,2024-03-01
3,204,Home,2023-04-01,2024-04-01


In [25]:
policies.isnull().sum()

Policy_ID      0
Policy_Type    0
Start_Date     0
End_Date       0
dtype: int64

DATA CLEANING = POLICIES DATA STARTS

In [26]:
for col in policies.select_dtypes(include="object").columns:
    policies[col] = policies[col].str.strip()

In [27]:
# Standardize column names
policies.columns = policies.columns.str.strip().str.lower().str.replace(" ", "_")

In [28]:
policies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   policy_id    4 non-null      int64 
 1   policy_type  4 non-null      object
 2   start_date   4 non-null      object
 3   end_date     4 non-null      object
dtypes: int64(1), object(3)
memory usage: 256.0+ bytes


In [29]:
# Convert date columns
policies["start_date"] = pd.to_datetime(policies["start_date"], errors="coerce")
policies["end_date"] = pd.to_datetime(policies["end_date"], errors="coerce")

In [30]:
# Remove duplicates
policies = policies.drop_duplicates()

In [31]:
policies["policy_id"].isna().sum()

np.int64(0)

In [32]:
# Remove rows with invalid policy_id
policies = policies.dropna(subset=["policy_id"])

In [33]:
print(policies.columns)


Index(['policy_id', 'policy_type', 'start_date', 'end_date'], dtype='object')


DATA MODELLING = POLICIES DATA STARTS

In [34]:
# Policy duration
policies["policy_duration_days"] = (policies["end_date"] - policies["start_date"]).dt.days

In [35]:
print(policies.head())


   policy_id policy_type start_date   end_date  policy_duration_days
0        201        Life 2023-01-01 2024-01-01                   365
1        202      Health 2023-02-01 2024-02-01                   365
2        203        Auto 2023-03-01 2024-03-01                   366
3        204        Home 2023-04-01 2024-04-01                   366


In [36]:
policies["policy_status"] = policies["end_date"].apply(
    lambda x: "Active" if x >= pd.Timestamp.today() else "Expired"
)


In [37]:
policies.head()

,policy_id,policy_type,start_date,end_date,policy_duration_days,policy_status
0,201,Life,2023-01-01,2024-01-01,365,Expired
1,202,Health,2023-02-01,2024-02-01,365,Expired
2,203,Auto,2023-03-01,2024-03-01,366,Expired
3,204,Home,2023-04-01,2024-04-01,366,Expired


CLEANED AND MODELLED POLICIES DATA = EXPORT STARTS

In [38]:
policies.to_excel("outputs/DimPolicy.xlsx", index=False)


DATA PROFILLING = TRANSACTIONS DATA STARTS

In [39]:
transactions=pd.read_json("transactions.json")

In [40]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Transaction_ID  4 non-null      int64         
 1   Customer_ID     4 non-null      int64         
 2   Amount          4 non-null      int64         
 3   Date            4 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 256.0 bytes


In [41]:
transactions.head()

,Transaction_ID,Customer_ID,Amount,Date
0,301,101,250,2023-05-01
1,302,102,400,2023-05-03
2,303,103,150,2023-05-05
3,304,104,300,2023-05-07


DATA CLEANING = TRANSACTION DATA STARTS

In [42]:
transactions.columns = transactions.columns.str.strip().str.lower().str.replace(" ", "_")

In [43]:
for col in transactions.select_dtypes(include="object").columns:
    transactions[col] = transactions[col].str.strip()


In [44]:
transactions.isnull().sum()

transaction_id    0
customer_id       0
amount            0
date              0
dtype: int64

In [45]:
transactions["transaction_id"]=pd.to_numeric(transactions["transaction_id"],errors="coerce")
transactions["customer_id"]=pd.to_numeric(transactions["customer_id"],errors="coerce")
# Convert date
transactions["date"] = pd.to_datetime(transactions["date"], errors="coerce")

# Ensure numeric amount
transactions["amount"] = pd.to_numeric(transactions["amount"], errors="coerce")


In [46]:
transactions=transactions.drop_duplicates()

In [47]:
transactions.isnull().sum()


transaction_id    0
customer_id       0
amount            0
date              0
dtype: int64

CHECK FOR NOT APPLICABLE VALUE COUNT BEFORE DROPPING

In [48]:
transactions[
    ["transaction_id","customer_id","amount","date"]
].isna().sum()

transaction_id    0
customer_id       0
amount            0
date              0
dtype: int64

In [49]:
transactions = transactions.dropna(subset=["transaction_id", "customer_id", "amount", "date"])


AFTER DROPPING COUNT FOR DATA QUALITY CONTROL

In [50]:
transactions.isna().sum()


transaction_id    0
customer_id       0
amount            0
date              0
dtype: int64

DATA MODELLING = POLICIES DATA STARTS

In [65]:
final_df = customers.copy()

final_df = final_df.merge(txn_dates, on="customer_id", how="left")


In [66]:
#Derives the first and most recent transaction dates per customer using min and max aggregations.
txn_dates = transactions.groupby("customer_id")["date"].agg(
    first_transaction="min",
    last_transaction="max"
).reset_index()


In [70]:
final_df = final_df.merge(txn_dates, on="customer_id", how="left")


In [71]:
transactions.head()

,transaction_id,customer_id,amount,date
0,301,101,250,2023-05-01
1,302,102,400,2023-05-03
2,303,103,150,2023-05-05
3,304,104,300,2023-05-07


CLEANED AND MODELLED TRANSACTIONS DATA = EXPORT STARTS

In [72]:
transactions.to_excel("outputs/FactTransactions.xlsx", index=False)
final_df.to_excel("outputs/FactCustomerAnalytics.xlsx", index=False)


DATA PROFILLING = LOGS DATA STARTS

In [73]:
logs = pd.read_table("logs.txt")

In [74]:
logs.head()

,2023-05-01 INFO Customer 101 submitted claim
0,2023-05-02 ERROR Transaction failed for Custom...
1,2023-05-03 INFO Customer 103 premium paid
2,2023-05-04 WARNING Late claim submission by Cu...


In [75]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 1 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   2023-05-01 INFO Customer 101 submitted claim  3 non-null      object
dtypes: object(1)
memory usage: 152.0+ bytes


In [76]:
logs = pd.read_table("logs.txt", header=None, names=["raw"])

In [77]:
logs.head()

,raw
0,2023-05-01 INFO Customer 101 submitted claim
1,2023-05-02 ERROR Transaction failed for Custom...
2,2023-05-03 INFO Customer 103 premium paid
3,2023-05-04 WARNING Late claim submission by Cu...


DATA CLEANING AND MODELLING = LOGS DATA STARTS

In [78]:
def parse_log(row):
    parts = row.split(" ", 2)  # Split only on the first 2 spaces
    if len(parts) == 3:
        return pd.Series(parts, index=["date", "level", "message"])
    else:
        return pd.Series([None, None, None], index=["date", "level", "message"])

In [79]:
logs[["date", "level", "message"]] = logs["raw"].apply(parse_log)
logs = logs.drop(columns=["raw"])

In [80]:
logs["date"] = pd.to_datetime(logs["date"], errors="coerce")

In [81]:
# Extract customer_id from message
logs["customer_id"] = logs["message"].str.extract(r"Customer\s+(\d+)")
logs["customer_id"] = pd.to_numeric(logs["customer_id"], errors="coerce")

In [82]:
# Total logs per customer
log_count = logs.groupby("customer_id")["message"].count().reset_index()
log_count.rename(columns={"message": "log_count"}, inplace=True)

In [83]:
logs.head()

,date,level,message,customer_id
0,2023-05-01,INFO,Customer 101 submitted claim,101
1,2023-05-02,ERROR,Transaction failed for Customer 102,102
2,2023-05-03,INFO,Customer 103 premium paid,103
3,2023-05-04,WARNING,Late claim submission by Customer 104,104


In [84]:
# Error logs per customer
error_count = logs[logs["level"] == "ERROR"].groupby("customer_id")["message"].count().reset_index()
error_count.rename(columns={"message": "error_count"}, inplace=True)

In [85]:
logs.head()

,date,level,message,customer_id
0,2023-05-01,INFO,Customer 101 submitted claim,101
1,2023-05-02,ERROR,Transaction failed for Customer 102,102
2,2023-05-03,INFO,Customer 103 premium paid,103
3,2023-05-04,WARNING,Late claim submission by Customer 104,104


In [86]:
# First and last log dates
log_dates = logs.groupby("customer_id")["date"].agg(first_log="min", last_log="max").reset_index()

In [87]:
# Step 6: Merge features into customers
final_df = customers.merge(log_count, on="customer_id", how="left")
final_df = final_df.merge(error_count, on="customer_id", how="left")
final_df = final_df.merge(log_dates, on="customer_id", how="left")

In [88]:
final_df["has_claim"] = logs.groupby("customer_id")["message"].apply(lambda x: x.str.contains("claim", case=False).any()).reset_index(drop=True)


In [89]:
final_df["has_payment"] = logs.groupby("customer_id")["message"].apply(lambda x: x.str.contains("paid|payment|premium", case=False).any()).reset_index(drop=True)


In [90]:
final_df["days_since_last_log"] = (pd.Timestamp.today() - final_df["last_log"]).dt.days


In [91]:
# Step 7: Check final DataFrame
final_df.head()

,customer_id,name,premium_amount,claim_amount,profit,claim_ratio,risk_level,premium_category,log_count,error_count,first_log,last_log,has_claim,has_payment,days_since_last_log
0,101,Alice,1000,200,800,0.200000,Low Risk,Standard,1,NaN,2023-05-01,2023-05-01,True,False,992
1,102,Bob,1500,500,1000,0.333333,Medium Risk,High_Value,1,1.0,2023-05-02,2023-05-02,False,False,991
2,103,Charlie,1200,0,1200,0.000000,Low Risk,Standard,1,NaN,2023-05-03,2023-05-03,False,True,990
3,104,David,1300,300,1000,0.230769,Low Risk,Standard,1,NaN,2023-05-04,2023-05-04,True,False,989


CLEANED AND MODELLED LOGS DATA = EXPORT STARTS

In [92]:
final_df.to_excel("outputs/FactCustomerLogAnalysis.xlsx", index=False)